The code in this notebook has been copied from [here.](https://pytorch-tutorial.readthedocs.io/en/latest/tutorial/chapter03_intermediate/3_2_2_cnn_resnet_cifar10/)

In [8]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import random_split

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
device

device(type='cuda')

In [32]:
# Hyper-parameters
num_epochs = 50
learning_rate = 0.001

In [5]:
# Image preprocessing modules
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

In [6]:
# CIFAR-10 dataset
dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=False, 
                                            transform=transforms.ToTensor())


Extracting ../../data/cifar-10-python.tar.gz to ../../data/


In [9]:
train_size = 45000
val_size = 5000

train_ds, val_ds = random_split(dataset, [train_size, val_size])

In [12]:
train_loader = torch.utils.data.DataLoader(dataset=train_ds,
                                           batch_size=100, 
                                           shuffle=True)

val_loader = torch.utils.data.DataLoader(dataset=val_ds, 
                                         batch_size=100, 
                                         shuffle=False)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100, 
                                          shuffle=False)

In [13]:
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, bias=False)

In [14]:
# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out


In [15]:
# ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)

    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None

        #in case output channels and input channels do not match, we need an extra function to downsample x in order to match the output dimensions
        #downsampling is also required when the stride is not equal to 1 (since the image height and width is decreased)

        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
            
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [33]:
model = ResNet(ResidualBlock, [10, 10, 10]).to(device) # a total of 32 layers

In [34]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [35]:
# For updating learning rate
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [36]:
def get_accuracy(loader):
  model.eval()
  correct = 0
  total = 0
  with torch.no_grad():
    for images, labels in loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

  model.train()

  return correct/total

In [37]:
# Train the model
total_step = len(train_loader)
curr_lr = learning_rate
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 50 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
          
    acc_t = get_accuracy(train_loader)
    acc_v = get_accuracy(val_loader)
    print ("Epoch [{}/{}] Training Accuracy: {:.4f} Val Accuracy: {:.4f}"
                   .format(epoch+1, num_epochs, acc_t, acc_v))
    

    
    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)


Epoch [1/50], Step [50/450] Loss: 1.8708
Epoch [1/50], Step [100/450] Loss: 1.7392
Epoch [1/50], Step [150/450] Loss: 1.8269
Epoch [1/50], Step [200/450] Loss: 1.6855
Epoch [1/50], Step [250/450] Loss: 1.5257
Epoch [1/50], Step [300/450] Loss: 1.6436
Epoch [1/50], Step [350/450] Loss: 1.3280
Epoch [1/50], Step [400/450] Loss: 1.2940
Epoch [1/50], Step [450/450] Loss: 1.3824
Epoch [1/50] Training Accuracy: 0.4215 Val Accuracy: 0.4130
Epoch [2/50], Step [50/450] Loss: 1.2738
Epoch [2/50], Step [100/450] Loss: 1.2290
Epoch [2/50], Step [150/450] Loss: 1.3953
Epoch [2/50], Step [200/450] Loss: 1.2054
Epoch [2/50], Step [250/450] Loss: 1.2379
Epoch [2/50], Step [300/450] Loss: 1.1207
Epoch [2/50], Step [350/450] Loss: 1.2765
Epoch [2/50], Step [400/450] Loss: 1.2269
Epoch [2/50], Step [450/450] Loss: 1.1064
Epoch [2/50] Training Accuracy: 0.5786 Val Accuracy: 0.5772
Epoch [3/50], Step [50/450] Loss: 1.0563
Epoch [3/50], Step [100/450] Loss: 0.9344
Epoch [3/50], Step [150/450] Loss: 1.2028
E

In [39]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the train images: {:.2f} %'.format(100 * correct / total))

    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {:.2f} %'.format(100 * correct / total))


Accuracy of the model on the train images: 98.08 %
Accuracy of the model on the test images: 88.55 %
